In [2]:
##Commencement : importer les packages

import requests 
import pandas as pd
import json   
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import re

### COMMENCEMENT DU PROGRAMME : Webscrapping sur *AUTOSCOUT24*
importer les packages 

In [3]:
website = "https://www.autoscout24.fr/lst/mercedes-benz?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=F&search_id=cw4927j1c&page=1"

In [4]:
response = requests.get(website)

In [5]:
response.status_code

200

In [6]:
#soup object

soup = BeautifulSoup(response.content,"html.parser")

In [7]:
#resultat
 
results = soup.find_all("article",{"class" : "cldt-summary-full-item listing-impressions-tracking list-page-item ListItem_article__ppamD" })

In [8]:
len(results)

20

TARGER NECESSARY 

- modele 
- prix du vehicule 
- annee de sortie 
- CO2
- info sur seller 
- nombre de revu 

### Generer une boucle et cibler les balises et classes du code source 

In [9]:
name = []
prix = []
info = []
sell = []

for i in range(1,30):
    
    # website dans une variable
    website = "https://www.autoscout24.fr/lst/mercedes-benz?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=F&search_id=cw4927j1c&page="+str(i)

    # resquest au website
    response = requests.get(website)
    
    # soup object
    soup = BeautifulSoup(response.content,"html.parser")
    
    #result
    results = soup.find_all("article",{"class" : "cldt-summary-full-item listing-impressions-tracking list-page-item ListItem_article__ppamD"})
    
    
    for result in results :
        try:
            prix.append(result.find("p",{"class":"Price_price__WZayw"}).get_text())
        except:
            prix.append('n/a')
        try:
            name.append(result.find("h2").get_text())
        except:
            name.append('n/a')  
        try:
            info.append(result.find("div",{"class":"VehicleDetailTable_container__mUUbY"}).get_text())
        except:
            info.append('n/a')  
        try:
            sell.append(result.find("span",{"class":"SellerInfo_name__yjUE6"}).get_text())
        except:
            sell.append('n/a')



#### Nettoyage de données : 

In [10]:
car_dealer = pd.DataFrame({"Modele":name , "prix_en_euro":prix, "info vehicule":info, "Info vendeur" : sell })

In [11]:
car_dealer

,Modele,prix_en_euro,info vehicule,Info vendeur
0,Mercedes-Benz CLS 53 AMG,"€ 69 988,-",23 500 km08/2018320 kW (435 CH)Occasion- (Prop...,Tabor Mobile(79)
1,Mercedes-Benz CL,n/a,47 000 km02/2015120 kW (163 CH)Occasion- (Prop...,PLANETE AUTO(1)
2,Mercedes-Benz E 200,"€ 34 959,-",48 035 km02/2019135 kW (184 CH)Occasion- (Prop...,Tabor Mobile(79)
3,Mercedes-Benz C 220,"€ 6 990,-",90 000 km01/2002105 kW (143 CH)Occasion1 Propr...,JESSICAR 2020
4,Mercedes-Benz CLS 350,n/a,92 000 km03/2018210 kW (286 CH)Occasion1 Propr...,Center Car
...,...,...,...,...
375,Mercedes-Benz CL,"€ 21 900,-",43 176 km12/2019- kW (- CH)Occasion- (Propriét...,MERCEDES MEAUX - AUTOSPHERE
376,Mercedes-Benz CL,"€ 26 999,-",41 312 km06/2019- kW (- CH)Occasion- (Propriét...,PEUGEOT VICHY - AUTOSPHERE(1)
377,Mercedes-Benz 190,"€ 109 990,-",76 000 km01/195677 kW (105 CH)Occasion- (Propr...,DRIVE Automobiles
378,Mercedes-Benz A 180,"€ 26 990,-",64 500 km02/202085 kW (116 CH)Occasion- (Propr...,sasu cg canteleu(1)


In [12]:
car_dealer["Nombre kilometre"] = car_dealer["info vehicule"].str.split("km", expand = True)[0]

In [13]:
car_dealer["Taux_CO2"] = car_dealer["info vehicule"].str.split("km", expand = True)[2]

In [14]:
car_dealer["annee"] = car_dealer["info vehicule"].str.split('km', expand= True)[1]

In [15]:
def first_7_chars(string):
    return string[:7]

In [16]:
car_dealer["annee_sortie"]=car_dealer["annee"].apply(first_7_chars)

In [17]:
def caract_entre(between):
    return between[7:22]

In [18]:
car_dealer["puissance"] = car_dealer["annee"].apply(caract_entre)

In [19]:
def delete_first_30_chars(string):
    return string[20:]

In [20]:
car_dealer["annee"] = car_dealer["annee"].apply(delete_first_30_chars)

In [21]:
def delete_first_10_chars(string):
    return string[10:]

In [22]:
car_dealer["annee"] = car_dealer["annee"].apply(delete_first_10_chars)

In [23]:
def delete_2_last_chars(string):
    return string[:1]

In [24]:
def keep_digits(string):
    digits = ''.join(filter(str.isdigit, string))
    return pd.to_numeric(digits)

In [25]:
car_dealer['Taux_CO2'] = car_dealer['Taux_CO2'].apply(keep_digits)

In [26]:
car_dealer['prix_en_euro'] = car_dealer['prix_en_euro'].apply(keep_digits)

In [70]:
def extract_words(string, words_list):
    words = []
    for word in words_list:
        if word in string:
            words.append(word)
    return words

In [71]:
car_dealer['Transmission'] = car_dealer['annee'].apply(lambda x: extract_words(x, ["Semi-automatique","automatique", "manuelle"]))

In [72]:
car_dealer['Consommation'] = car_dealer['annee'].apply(lambda x: extract_words(x, ["Diesel","Essence", "Électrique/Essence","Électrique"]))

In [73]:
del car_dealer["annee"]

In [74]:
del car_dealer["info vehicule"]

In [75]:
## utilisation de regex pour extraire les informations d'une parenthese 
car_dealer['puissance'] = car_dealer['puissance'].str.extract(r"\((.*)\)")

In [76]:
car_dealer

,Modele,prix_en_euro,Info vendeur,Nombre kilometre,Taux_CO2,annee_sortie,puissance,Transmission,Consommation
0,Mercedes-Benz AMG GT,109990.0,SASU,45 000,224.0,04/2016,510 CH,[automatique],[Essence]
1,Mercedes-Benz S 63 AMG,179900.0,PARIS PRESTIGE CARS(9),8 981,253.0,10/2019,612 CH,[automatique],[Essence]
2,Mercedes-Benz CLK,15990.0,Agence Automobilière de Senlis(2),85 000,276.0,10/2002,306 CH,[automatique],[]
3,Mercedes-Benz G 63 AMG,899500.0,PARIS PRESTIGE CARS(9),315,NaN,12/2021,799 CH,[automatique],[Essence]
4,Mercedes-Benz S 320,14990.0,PACK AUTOS,47 000,266.0,08/2002,223 CH,[automatique],[Essence]
...,...,...,...,...,...,...,...,...,...
395,Mercedes-Benz CL,24990.0,MERCEDES LA ROCHELLE - AUTOSPHERE(3),85 270,NaN,12/2016,- CH,[automatique],[Diesel]
396,Mercedes-Benz CL,21900.0,MERCEDES MEAUX - AUTOSPHERE,43 176,NaN,12/2019,- CH,[manuelle],[Essence]
397,Mercedes-Benz CL,26999.0,PEUGEOT VICHY - AUTOSPHERE(1),41 312,NaN,06/2019,- CH,[automatique],[Diesel]
398,Mercedes-Benz 190,109990.0,DRIVE Automobiles,76 000,NaN,01/1956,105 CH,[manuelle],[Essence]


La base de données étant propre, nous pouvons désormais basculer sur un logiciel pour créer des visualisation

In [77]:
# exportation sous excel
#car_dealer.to_excel("autoscout_excel_2.xlsx", index= False)
#car_dealer.to_csv('car_dealer.csv',encoding='utf-8')

In [1]:
car_dealer['Nombre kilometre'] = car_dealer['Nombre kilometre'].str.strip()


NameError: name 'car_dealer' is not defined